In [ ]:
import pandas as pd
import re
import statsapi
from requests import get

import constants
import utils

## TODO: Take the sheet directly from Google Sheets instead of CSV

In [ ]:
pl_export = utils.load_dataframe(constants.OWNED_PLAYERS_FILENAME)

In [ ]:
owned_players = utils.clean_pl_export(pl_export)

In [ ]:
single_teams = []
for column in owned_players.columns:
    if column == 'Team Position':
        continue
    single_team = owned_players[['Team Position', column]].copy()
    single_team.columns = ['Team Position', 'Player']
    single_team['PL Owner'] = column
    single_teams.append(single_team)
len(single_teams)

In [ ]:
owned_players_full = pd.concat(single_teams)
# Remove empty bench spots
owned_players_full = owned_players_full.loc[~pd.isna(owned_players_full['Player'])]
owned_players_full

In [ ]:
owned_players_full[['PL Name', 'Pos Team']] = owned_players_full['Player'].str.rsplit('-', n=1, expand=True)
owned_players_full['Player Positions'] = owned_players_full['Pos Team'].str.strip().str[:-3]
owned_players_full['Team'] = owned_players_full['Pos Team'].str.strip().str[-3:].str.upper()
owned_players_full

In [ ]:
def get_player_id_from_pl_name(pl_name, pl_team):
    player_id = -1
    search_value = None

    # Split the PL-formatted name (JTRealmuto) on capital letters. Take the last value (name) unless special handling is required.
    split_name = re.findall('[A-Z][^A-Z]*', pl_name)
    first_initial = split_name[0][0]
    split_last_name_index = -1
    last_name = split_name[split_last_name_index]

    # Handle special cases...
    #if pl_name in constants.PLAYER_LOOKUP_VALUES.keys():
    #    search_value = constants.PLAYER_LOOKUP_VALUES[pl_name]
    search_value = last_name
    while search_value.upper() in ('JR', 'I'):
        split_last_name_index -= 1
        search_value = split_name[split_last_name_index]

    # Lookup player in statsapi
    lookup_results = get(f'{constants.STATSAPI_PEOPLE_SEARCH_PATH}?names={search_value}&hydrate=currentTeam').json()['people']

    print(f'Got {len(lookup_results)} results from the API for player {pl_name}, team {pl_team}, search value {search_value}')

    #if pl_name in constants.PLAYER_LOOKUP_VALUES.keys():
    #    player_id = next((player for player in lookup_results), {}).get('id')
    #    print(f'Handled special case for player {pl_name}, using search value {search_value}')
    #else:
    for player in lookup_results:
        team_id = utils.get_team_id(pl_team)
        if first_initial == player['fullName'][0] and team_id == player['currentTeam']['id']:
            player_id = player['id']
            break

    if player_id == -1:
        print(f'Could not match PL player {pl_name} to an ID, lookup results: {lookup_results}')

    return player_id

In [ ]:
owned_players_full['Player ID'] = owned_players_full.apply(lambda row: get_player_id_from_pl_name(row['PL Name'], row['Team']), axis=1)
owned_players_full

In [ ]:
utils.save_dataframe(owned_players_full, constants.OWNED_PLAYERS_IDS_FILENAME)